In [ ]:
%%sh
cat > pi-mpi.c < EOF
#include <stdio.h>
#include <stdlib.h>
#include <assert.h>
#include <string.h>
#include "mpi.h"

//# Deklaracja stałych, które określają liczbę iteracji, szerokość siatki i liczbę procesów.
#define DEFAULT_ITERATIONS 64
#define GRID_WIDTH 256
#define DIM 16 // 


//# Funkcje wyświetlające kolory

void red () {
  printf("\033[1;31m");
}
void Blue(){
  printf("\033[0;34m");
}
void reset () {
  printf("\033[0m");
}
//# funkcja licząca prawidłowo sąsiadów na krawędziach
int mod(int a, int b)
{
    int r = a % b;
    return r < 0 ? r + b : r;
}
int main(int argc, char **argv)
{
  double  suma[DEFAULT_ITERATIONS]  ; 
  clock_t tic = clock();
    int global_grid[256] =
        {0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
         0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
         1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 1 , 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
         0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
         0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0};
    int num_procs;
    int ID, j;
    int iters = 0;
    int num_iterations;
    //# Sprawdzanie liczby argumentów
    if (argc == 1)
    {
        num_iterations = DEFAULT_ITERATIONS;
    }
    else if (argc == 2)
    {
        num_iterations = atoi(argv[1]);
    }
    else
    {
        printf("Usage: ./gameoflife \n");
        exit(1);
    }
    //# inicjalizujemy MPI
    MPI_Status stat;
    if (MPI_Init(&argc, &argv) != MPI_SUCCESS)
    {
        printf("error\n");
    }
    //# Ustawianie ranku procesu
    MPI_Comm_size(MPI_COMM_WORLD, &num_procs);
    MPI_Comm_rank(MPI_COMM_WORLD, &ID);
    assert(DIM % num_procs == 0); //# sprawdza czy liczba kolumn siatki jest podzielna przez liczbę procesorów
    int *arr = (int *)malloc(DIM * ((DIM / num_procs) + 2) * sizeof(int)); //# tworzy tablicę i przypisuje do niej pamięć 
    for (iters = 0; iters < num_iterations; iters++) //# przeprowadza num_iterations iteracji symulacji , Przepisuje dane z globalnej tablicy global_grid do tablicy arr zaczynając
    {
        //printf("%d %d\n",ID, DIM * ((DIM / num_procs) + 2));
        j = DIM;
        for (int i = ID * (GRID_WIDTH / num_procs); i < (ID + 1) * (GRID_WIDTH / num_procs); i++)
        {
            arr[j] = global_grid[i];
            // if(ID==1)
            //     printf(" %d %d \n",j,i);
            j++;
        }
        //# Jeśli liczba procesorów jest większa niż 1, program przeprowadza odd-even send_recv, czyli nieparzyste procesory otrzymują dane od poprzedniego i następnego procesora, a parzyste procesory przesyłają swoje dane do poprzedniego i następnego procesora.
        if (num_procs != 1)
        {
            //# odd-even send_recv
            int incoming_1[DIM];
            int incoming_2[DIM];
            int send_1[DIM];
            int send_2[DIM];
            //# Jeśli ID procesu jest parzyste
            if (ID % 2 == 0)
            {
                //# Przypisanie do tablicy send_1 wierszy od DIM do DIM + i
                for (int i = 0; i < DIM; i++)
                {
                    send_1[i] = arr[i + DIM];
                    // printf(" - %d - ",send_1[i]);
                    // printf(" %d %d\n ",i,i+DIM);
                }
                //# Wysłanie tablicy send_1 do procesu o ID - 1 (z modulo, aby zapobiec przekroczeniu zakresu)
                MPI_Ssend(&send_1, DIM, MPI_INT, mod(ID - 1, num_procs), 1, MPI_COMM_WORLD);
                //# Przypisanie do tablicy send_2 ostatnich wierszy o indeksie (DIM * (DIM / num_procs)) + i
                for (int i = 0; i < DIM; i++)
                {
                    send_2[i] = arr[(DIM * (DIM / num_procs)) + i];
                    // printf(" %d %d\n ",i,(DIM * (DIM / num_procs)) + i);
                }
                //# Wysłanie tablicy send_2 do procesu o ID + 1 (z modulo, aby zapobiec przekroczeniu zakresu)
                MPI_Ssend(&send_2, DIM, MPI_INT, mod(ID + 1, num_procs), 1, MPI_COMM_WORLD);
            }
            //# Jeśli ID procesu jest nieparzyste
            else
            {
                //# Odbieranie tablicy incoming_2 od procesu o ID + 1 (z modulo, aby zapobiec przekroczeniu zakresu)
                MPI_Recv(&incoming_2, DIM, MPI_INT, mod(ID + 1, num_procs), 1, MPI_COMM_WORLD, &stat);
                //# Odbieranie tablicy incoming_1 od procesu o ID - 1 (z modulo, aby zapobiec przekroczeniu zakresu)
                MPI_Recv(&incoming_1, DIM, MPI_INT, mod(ID - 1, num_procs), 1, MPI_COMM_WORLD, &stat);
            }
            //# Jeśli ID procesu jest parzyste
            if (ID % 2 == 0)
            {
                //# Odbieranie tablicy incoming_2 od procesu o ID + 1 (z modulo, aby zapobiec przekroczeniu zakresu)
                MPI_Recv(&incoming_2, DIM, MPI_INT, mod(ID + 1, num_procs), 1, MPI_COMM_WORLD, &stat);
                //# Odbieranie tablicy incoming_1 od procesu o ID - 1 (z modulo, aby zapobiec przekroczeniu zakresu)
                MPI_Recv(&incoming_1, DIM, MPI_INT, mod(ID - 1, num_procs), 1, MPI_COMM_WORLD, &stat);
            }
            else
            {
                //# Wysłanie pierwszych 16 elementów z tablicy arr do procesu o ID - 1
                for (int i = 0; i < DIM; i++)
                {
                    send_1[i] = arr[i + DIM];
                }
                MPI_Ssend(&send_1, DIM, MPI_INT, mod(ID - 1, num_procs), 1, MPI_COMM_WORLD);

                //# Wysłanie ostatnich 16 elementów z tablicy arr do procesu o ID + 1
                for (int i = 0; i < DIM; i++)
                {
                    send_2[i] = arr[(DIM * (DIM / num_procs)) + i];
                }
                MPI_Ssend(&send_2, DIM, MPI_INT, mod(ID + 1, num_procs), 1, MPI_COMM_WORLD);
            }
            //# Przypisanie odebranych danych z tablic incoming_1 i incoming_2 do tablicy arr
            for (int i = 0; i < DIM; i++)
            {
                arr[i] = incoming_1[i];
                arr[(DIM * ((DIM / num_procs) + 1)) + i] = incoming_2[i];
            }
        }
        //# Jeśli identyfikator procesu nie jest parzysty
        else
        {
            //# Przypisanie danych z tablicy
            for (int i = 0; i < DIM; i++)
            {
                arr[i + GRID_WIDTH + DIM] = global_grid[i];
                //printf(" %d %d \n",i + GRID_WIDTH+DIM,i);
            }
            for (int i = GRID_WIDTH; i < GRID_WIDTH + DIM; i++)
            {
                arr[i - GRID_WIDTH] = global_grid[i - DIM];
                //printf(" %d %d \n",i - GRID_WIDTH,i-DIM);
            }
        }
       //# zasady gry
        int * final = (int *)malloc(DIM * ((DIM / num_procs)) * sizeof(int));

        //# Pętla for przegląda każdy element w planszy
        for (int k = DIM; k < DIM * ((DIM / num_procs) + 1); k++)
        {
            //# Oblicz liczbę wierszy i kolumn w planszy
            int total_rows = DIM * (DIM / num_procs) + 2;
            int r = k / DIM;
            int c = k % DIM;

            //# Znajdź pozycję poprzedniego i następnego elementu w wierszu i kolumnie (c-kolumna, r-wiersz)
            int prev_r = mod(r - 1, total_rows);
            int prev_c = mod(c - 1, DIM);
            int next_r = mod(r + 1, total_rows);
            int next_c = mod(c + 1, DIM);
            //# Zlicz liczbę żywych komórek wokół bieżącej komórki
            int count = arr[prev_r * DIM + prev_c] + arr[prev_r * DIM + c] + arr[prev_r * DIM + next_c] + arr[r * DIM + prev_c] + arr[r * DIM + next_c] + arr[next_r * DIM + prev_c] + arr[next_r * DIM + c] + arr[next_r * DIM + next_c];
            //# Zastosuj reguły gry w życie, aby określić stan następnej komórki
            if (arr[k] == 1)
            {
                //# Jeśli komórka jest żywa i ma mniej niż 2 lub więcej niż 3 żywe sąsiednie komórki, zostanie ona zabita
                if (count < 2)
                    final[k - DIM] = 0;
                else if (count > 3)
                    final[k - DIM] = 0;
                else
                    final[k - DIM] = 1;
            }
            else
            {
                //# Jeśli komórka jest martwa i ma dokładnie 3 żywe sąsiednie komórki, zostanie ona ożywiona
                if (count == 3)
                    final[k - DIM] = 1;
                else
                    final[k - DIM] = 0;
            }
        }

        j = 0;
        //# Pętla przeglądająca elementy w tablicy "final" i przypisująca je do odpowiadających im miejsc w tablicy "global_grid"
        for (int i = ID * (GRID_WIDTH / num_procs); i < (ID + 1) * (GRID_WIDTH / num_procs); i++)
        {
            global_grid[i] = final[j];
            j++;
        }
        //# Funkcja MPI_Gather gromadząca dane z wielu procesów i zapisująca je w tablicy "global_grid" na procesie z ID = 0
        MPI_Gather(final, DIM * (DIM / num_procs), MPI_INT, &global_grid, DIM * (DIM / num_procs), MPI_INT, 0, MPI_COMM_WORLD);

        //# wypisanie iteracji
        if (ID == 0)
        {
            printf("\n %d: przejście:\n", iters);
            for (j = 0; j < GRID_WIDTH; j++)
            {
                if (j % DIM == 0)
                {
                    printf("\n");
                    //red();
                }
                int temp = global_grid[j] + 3 ;
                if(temp == 4){
                  red();
                } 

                // GLOWNE WYSWIETLANIE !!!!!!!!
                printf("%d  ", global_grid[j] + 3 );
                Blue();
            }
            printf("\n");
        }
    }
    free(arr);
    MPI_Finalize(); 
    clock_t toc = clock();
    suma[j] = (toc - tic) / CLOCKS_PER_SEC ; 
    printf("czas trwania: %f s\n", (double)(toc - tic) / CLOCKS_PER_SEC);
   
}
EOF
mpicc pi-mpi.c && mpirun -n 16 --allow-run-as-root a.out